In [1]:
import copy
import os.path
import sys

In [2]:
os.getcwd()

'/home/Minwoo/LESN/Decompose/DecomposeBERT/experiment'

In [3]:
sys.path.append('/home/Minwoo/LESN/Decompose/DecomposeBERT/')

In [4]:
from utils.model_utils.evaluate import evaluate_model, test_f1, test

from utils.model_utils.load_model import *
from utils.model_utils.model_config import ModelConfig
from utils.dataset_utils.load_dataset import load_dataset
from utils.decompose_utils.weight_remover import WeightRemoverBert
from utils.decompose_utils.concern_identification import ConcernIdentificationBert
from utils.decompose_utils.tangling_identification import TanglingIdentification
from transformers import AutoConfig
from tqdm import tqdm
from datetime import datetime
from pprint import pp
from utils.decompose_utils.sampling import sampling_class
import torch

2024-04-09 15:33:24.070219: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 15:33:24.127074: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 15:33:24.859818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/Minwoo/.conda/envs/DecomposeBERT/lib/python3.8/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree

In [5]:
# model_name = "prajjwal1/bert-tiny"
# model_dir = "tiny bert"
model_name = "bert-base-uncased"
model_dir = "bert"
data = "SDG"
num_labels = 16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
checkpoint_name = "best_model.pt"
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model_config = ModelConfig(
    _model_name=model_name,
    _model_dir=model_dir,
    _data=data,
    _is_pretrained=True,
    _transformer_config=config,
    _checkpoint_name=checkpoint_name,
    _device=device,
)
model, tokenizer, checkpoint = load_classification_model(model_config)

train_dataloader, valid_dataloader, test_dataloader = load_dataset(
    model_config, tokenizer, batch_size=32, test_size=0.3
)
# print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
# evaluate_model(model, model_config, test_dataloader)

Loading dataset
Loading preprocessed data...
Loading preprocessed data...
Loading preprocessed data...


In [ ]:
for k in range(3):
    i = 0
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

#Module 0 in progress....
origin
0
{'precision': 0.751004016064257,
 'recall': 0.6824817518248175,
 'f1-score': 0.7151051625239006,
 'support': 274.0}
Start Positive CI sparse
1
{'precision': 0.7362204724409449,
 'recall': 0.6824817518248175,
 'f1-score': 0.7083333333333333,
 'support': 274.0}
2


In [ ]:
for k in range(3):
    i = 1
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

In [ ]:
for k in range(3):
    i = 2
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")

In [ ]:
for k in range(3):
    i = 3
    print("#Module " + str(i) + " in progress....")
    num_samples = 64

    positive_samples1 = sampling_class(
        train_dataloader, i, 20, num_labels, True, 4, device=device
    )
    positive_samples2 = sampling_class(
        train_dataloader, i, num_samples, num_labels, True, 4, device=device
    )
    negative_samples = sampling_class(
        train_dataloader, i, num_samples, num_labels, False, 4, device=device
    )

    module1 = copy.deepcopy(model)
    w = WeightRemoverBert(model, p=0.9)
    ci1 = ConcernIdentificationBert(model, p=0.6)
    ti1 = TanglingIdentification(model, p=0.9)

    ff1 = [
        [
            torch.sum(
                model.bert.encoder.layer[num].intermediate.dense.weight != 0
            ).item()
        ]
        for num in range(config.num_hidden_layers)
    ]
    ff2 = [
        [torch.sum(model.bert.encoder.layer[num].output.dense.weight != 0).item()]
        for num in range(config.num_hidden_layers)
    ]
    pooler = [torch.sum(model.bert.pooler.dense.weight != 0).item()]
    print("origin")
    j = 0
    print(j)
    results = test_f1(model, test_dataloader, model_config, False)
    pp(results["details"][f"{i}"])

    print("Start Positive CI sparse")

    for batch in positive_samples1:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = w.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Positive CI after sparse")

    for batch in positive_samples2:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t1 = ci1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    print("Start Negative TI")

    for batch in negative_samples:
        input_ids, attn_mask, _, total_sampled = batch
        with torch.no_grad():
            t = ti1.propagate(module1, input_ids)
        for num in range(config.num_hidden_layers):
            ff1[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].intermediate.dense.weight != 0
                ).item()
            )
            ff2[num].append(
                torch.sum(
                    module1.bert.encoder.layer[num].output.dense.weight != 0
                ).item()
            )
        pooler.append(torch.sum(module1.bert.pooler.dense.weight != 0).item())

        j+=1
        print(j)
        
        results = test_f1(module1, test_dataloader, model_config, False)
        pp(results["details"][f"{i}"])        

    for num in range(config.num_hidden_layers):
        print(f"f({ff1[num]})")
        print("\n")
    for num in range(config.num_hidden_layers):
        print(f"f({ff2[num]})")
        print("\n")
    print(f"f({pooler})")
    print("\n")